In [ ]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load data (replace path as needed)
df = pd.read_csv("/kaggle/input/hotel-booking-demand/hotel_bookings.csv")


df.shape, df.columns.tolist()
df.head(3)
df.info()
df.describe(include='all').T



**Data Cleaning**

In [ ]:
# check missing values
df.isna().sum().sort_values(ascending=False).head(10)


Handle missing:

In [ ]:
# Replace missing 'children' with 0
df['children'] = df['children'].fillna(0)

# Replace missing 'agent' and 'company' with 0 and convert to int
df['agent'] = df['agent'].fillna(0).astype(int)
df['company'] = df['company'].fillna(0).astype(int)

# Replace missing 'country' with 'Unknown'
df['country'] = df['country'].fillna('Unknown')

# verify
df.isna().sum().sum()


2.1 Convert dates & create arrival_date

In [ ]:
# month to number
month_map = {m: i for i, m in enumerate(['January','February','March','April','May','June',
                                         'July','August','September','October','November','December'], 1)}
df['arrival_date_month_num'] = df['arrival_date_month'].map(month_map)

# combine into one date
df['arrival_date'] = pd.to_datetime(dict(
    year=df['arrival_date_year'],
    month=df['arrival_date_month_num'],
    day=df['arrival_date_day_of_month']
))

# convert reservation_status_date
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])


Handle Duplicates

In [ ]:
# check duplicates
dups = df.duplicated().sum()
print("Duplicate rows:", dups)

# drop duplicates if found
df = df.drop_duplicates()


Create Helpful Derived Columns

In [ ]:
# total nights and total guests
df['total_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
df['total_guests'] = df['adults'] + df['children'] + df['babies']

# approximate total revenue per booking
df['total_revenue'] = df['adr'] * df['total_nights']

# lead time bucket
df['lead_time_bucket'] = pd.cut(df['lead_time'],
                                bins=[-1,7,30,90,180,365,10000],
                                labels=['<1w','1w-1m','1m-3m','3m-6m','6m-1y','>1y'])


EXPLORATORY DATA ANALYSIS (EDA)

Distribution of Bookings by Hotel Type

In [ ]:
sns.countplot(data=df, x='hotel', palette='pastel')
plt.title("Distribution of Bookings by Hotel Type")
plt.xlabel("Hotel Type")
plt.ylabel("Number of Bookings")
plt.show()

# % share
df['hotel'].value_counts(normalize=True)*100


Bookings by Month

In [ ]:
sns.countplot(data=df, x='arrival_date_month', order=month_map.keys(), palette='viridis')
plt.title("Bookings by Month")
plt.xticks(rotation=45)
plt.show()


Bookings by Market Segment

In [ ]:
sns.countplot(data=df, x='market_segment', palette='muted')
plt.title("Bookings by Market Segment")
plt.xticks(rotation=45)
plt.show()

df['market_segment'].value_counts(normalize=True)*100


Correlation between Lead Time and Cancellation Rate

In [ ]:
# Scatter plot
sns.scatterplot(data=df, x='lead_time', y='is_canceled', alpha=0.3)
plt.title("Lead Time vs Cancellation")
plt.show()

# numeric correlation
corr = df['lead_time'].corr(df['is_canceled'])
print("Correlation between lead time and cancellation:", corr)


Trend of Average Daily Rate (ADR) Over Time

In [ ]:
# average ADR per month-year
adr_trend = df.groupby(['arrival_date_year','arrival_date_month_num'])['adr'].mean().reset_index()
adr_trend['YearMonth'] = pd.to_datetime(adr_trend['arrival_date_year'].astype(str) + '-' + adr_trend['arrival_date_month_num'].astype(str) + '-01')

plt.figure(figsize=(10,5))
sns.lineplot(data=adr_trend, x='YearMonth', y='adr')
plt.title("Trend of Average Daily Rate (ADR) Over Time")
plt.xlabel("Month")
plt.ylabel("Average Daily Rate")
plt.show()


Frequency of Special Requests and Impact on Cancellations

In [ ]:
# distribution
sns.countplot(data=df, x='total_of_special_requests', palette='Set2')
plt.title("Frequency of Special Requests")
plt.show()

# cancellation rate by number of requests
cancel_by_req = df.groupby('total_of_special_requests')['is_canceled'].mean()
cancel_by_req.plot(kind='bar', color='skyblue')
plt.title("Cancellation Rate by Number of Special Requests")
plt.ylabel("Cancellation Rate")
plt.show()


Question 1: What is the average lead time for bookings?

In [ ]:
# Calculate the average lead time
avg_lead_time = df['lead_time'].mean()
median_lead_time = df['lead_time'].median()

print("Average Lead Time:", round(avg_lead_time, 2), "days")
print("Median Lead Time:", round(median_lead_time, 2), "days")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,4))
sns.histplot(df['lead_time'], bins=50, kde=True, color='skyblue')
plt.title("Distribution of Lead Time for Bookings")
plt.xlabel("Lead Time (days)")
plt.ylabel("Number of Bookings")
plt.show()


Interpretation

On average, guests book their stay about 80 days (~2.5 months) in advance.

The median being 49 days suggests that most customers book within 1–2 months, but some book far in advance (which raises the average).

This skewed distribution indicates a small group of early planners (possibly corporate or event-related bookings) while the majority are short-term leisure travelers.

**Quetion 2 -What is the distribution of bookings by hotel type?**

In [ ]:
# Count the number of bookings by hotel type
hotel_counts = df['hotel'].value_counts()

print(hotel_counts)

# Percentage share
hotel_percent = df['hotel'].value_counts(normalize=True) * 100
print("\nPercentage distribution:\n", hotel_percent)


**Visualization**

In [ ]:


plt.figure(figsize=(6,4))
sns.barplot(x=hotel_counts.index, y=hotel_counts.values, palette="viridis")

plt.title("Distribution of Bookings by Hotel Type")
plt.xlabel("Hotel Type")
plt.ylabel("Number of Bookings")
plt.show()


EDA Result: Distribution of Bookings by Hotel Type

City Hotel: 53,428 bookings (61.13%)
Resort Hotel: 33,968 bookings (38.87%)


*InterpretationThe majority of bookings (≈61%) are for City Hotels, suggesting a stronger preference for urban or business-related stays.Resort Hotels account for ≈39%, which is still substantial — often reflecting leisure, family vacations, or seasonal travel.This balance shows that while the business market dominates, the leisure segment also plays an important complementary role.*


Question3: How many bookings were canceled?

In [ ]:
# Count total cancellations
total_canceled = df['is_canceled'].sum()

# Count total bookings
total_bookings = df['is_canceled'].count()

# Calculate cancellation rate
cancel_rate = (total_canceled / total_bookings) * 100

print("Total Bookings:", total_bookings)
print("Total Canceled:", total_canceled)
print("Cancellation Rate:", round(cancel_rate, 2), "%")


In [ ]:


plt.figure(figsize=(5,5))
plt.pie([total_canceled, total_bookings - total_canceled],
        labels=["Canceled", "Not Canceled"],
        autopct='%1.1f%%', startangle=90,
        colors=["#ff6666","#66b3ff"])
plt.title("Booking Cancellation Distribution")
plt.show()


Total Bookings: 87396
Total Canceled: 24025
Cancellation Rate: 27.49 %


Interpretation

About 38% of all bookings were canceled — a significant proportion.

This shows that more than 1 in 3 customers who initially booked didn’t complete their stay.

High cancellation rates can affect revenue forecasts, inventory management, and staff scheduling.


Conclusion

Your data shows a noticeable cancellation trend (e.g., ~38%).

This directly affects revenue planning and resource allocation.

A mix of better policy design, customer segmentation, and targeted retention strategies can reduce cancellations and stabilize occupancy.

**Question4: What is the most common arrival month for bookings?**

In [ ]:
# Count the number of bookings per month
month_counts = df['arrival_date_month'].value_counts()

# Identify the most common month
most_common_month = month_counts.idxmax()
most_common_count = month_counts.max()

print("Bookings by Month:")
print(month_counts)
print("\nMost Common Arrival Month:", most_common_month, "with", most_common_count, "bookings")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.barplot(x=month_counts.index, y=month_counts.values, palette='Blues_d')
plt.title("Distribution of Bookings by Month", fontsize=14)
plt.xlabel("Month")
plt.ylabel("Number of Bookings")
plt.xticks(rotation=45)
plt.show()


**Most Common Arrival Month: August with 11257 bookings
Interpretation:

August represents the peak month for hotel arrivals, indicating the highest guest traffic during this time.

This aligns with summer holiday seasons in many regions, when both domestic and international travelers prefer vacation stays.
Conclusion:

August is the busiest month for Elite Hotels International.

Strategic actions during this period can maximize occupancy and revenue, while targeted marketing during off-peak months can balance seasonal fluctuations.

**Question5: What is the average number of special requests per booking?**

In [ ]:
# Calculate average and median special requests
avg_special_requests = df['total_of_special_requests'].mean()
median_special_requests = df['total_of_special_requests'].median()

# Distribution of requests
request_counts = df['total_of_special_requests'].value_counts().sort_index()

print("Average Special Requests per Booking:", round(avg_special_requests, 2))
print("Median Special Requests per Booking:", median_special_requests)
print("\nDistribution of Special Requests:\n", request_counts)


In [ ]:


plt.figure(figsize=(8,5))
sns.countplot(x='total_of_special_requests', data=df, palette='coolwarm')
plt.title("Distribution of Special Requests per Booking")
plt.xlabel("Number of Special Requests")
plt.ylabel("Count of Bookings")
plt.show()


Interpretation

The average of 0.7 indicates that on average, guests make less than one request per booking.

The median of 0.0 means that more than half of the guests don’t make any special requests at all.

However, a significant minority (~40%) of bookings include at least one request, which points to a segment of customers valuing personalization and comfort.

**Question6: Which country has the highest number of bookings?**

In [ ]:
# Count bookings by country
country_counts = df['country'].value_counts()

# Identify the top country
top_country = country_counts.idxmax()
top_country_count = country_counts.max()

print("Top Country:", top_country)
print("Number of Bookings:", top_country_count)
print("\nTop 10 Countries by Bookings:\n", country_counts.head(10))


In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=country_counts.head(10).index, y=country_counts.head(10).values, palette="crest")
plt.title("Top 10 Countries by Number of Bookings", fontsize=14)
plt.xlabel("Country")
plt.ylabel("Number of Bookings")
plt.xticks(rotation=45)
plt.show()

Interpretation:

Portugal (PRT) clearly leads with 27,453 bookings, showing that a large proportion of guests are domestic travelers.

The next top contributors — UK, France, Spain, and Germany — are all European countries, indicating a strong regional customer base.

This aligns with the European leisure and business travel market, which dominates the hotel’s clientele.

Conclusion:

Portugal (PRT) contributes the highest number of bookings — 27,453, confirming that domestic and nearby European markets are the key customer segments.

Focusing marketing resources on these regions while strategically promoting to new international markets can increase occupancy and diversify revenue.

**Question 7: What is the average daily rate (ADR) for each hotel type?**

In [ ]:
# Calculate average ADR by hotel type
avg_adr_by_hotel = df.groupby('hotel')['adr'].mean().round(2)
print(avg_adr_by_hotel)


hotel Type        Average ADR
City Hotel      110.99
Resort Hotel     99.03
Interpretation:

City Hotels have a higher average daily rate (€110.99) than Resort Hotels (€99.03).

This suggests that urban stays are priced more premium, likely due to:

Business travelers and short-term stays driving higher willingness to pay.

City locations generally having higher operating costs and demand during weekdays.

Resort Hotels, on the other hand, attract longer leisure stays, often booked in advance with discounted or package pricing.

Conclusion:

City Hotels: Average ADR = €110.99 → higher nightly revenue driven by short-term, business-oriented demand.

Resort Hotels: Average ADR = €99.03 → slightly lower, but potential to increase through value-added services and extended-stay promotions.


**Question 8: What percentage of guests required car parking spaces?**

In [ ]:
# Calculate the percentage of bookings requiring parking spaces
total_bookings = len(df)
bookings_with_parking = df[df['required_car_parking_spaces'] > 0].shape[0]
parking_percentage = (bookings_with_parking / total_bookings) * 100

print(f"Bookings with parking: {bookings_with_parking}")
print(f"Total bookings: {total_bookings}")
print(f"Percentage requiring parking: {parking_percentage:.2f}%")


Result:
Bookings with parking: 7313
Total bookings: 87396
Percentage requiring parking: 8.37%
Interpretation:

About 8.4% of total bookings requested car parking spaces, meaning that roughly 1 in every 12 guests arrives with their own vehicle.

This indicates moderate demand for parking facilities — not dominant, but still important for a specific segment of guests.

The parking demand likely varies by hotel type and location:

Resort Hotels → higher parking usage (families, road trips, vacationers).

City Hotels → lower parking usage (public transport, business travelers, taxis).

Conclusion:

8.37% of guests required car parking spaces, indicating moderate parking demand overall.

Actionable insight: Optimize the number of parking spots — focus on Resort Hotels and long-stay travelers, while minimizing space in City Hotels to save operational costs.

**Question10: What is the average stay duration in week nights and weekend nights?**

In [ ]:
bookings_via_agents = df[df['agent'] != 0].shape[0]
percentage_via_agents = (bookings_via_agents / len(df)) * 100

print(f"Bookings via agents: {bookings_via_agents}")
print(f"Percentage via agents: {percentage_via_agents:.2f}%")


Interpretation:

A very high proportion (86%) of bookings were made through travel agents, while only about 14% came from direct or other channels.

This suggests that Elite Hotels International relies heavily on intermediaries for acquiring customers.

Such dependence can have both benefits (steady volume) and risks (lower margins, limited customer data control).
Conclusion:

Bookings via agents: 75,203 (≈86.05%)

Insight: The majority of guests book through travel agencies, indicating strong partnerships but also high reliance on intermediaries.

Business Impact: Elite Hotels should maintain agent collaborations for consistent occupancy, while strategically boosting direct booking channels to improve margins and brand control.

**Medium Level Questions**

**Quetion 1-What is the cancellation rate for each hotel type?**

In [ ]:
df.groupby('hotel')['is_canceled'].mean() * 100


Conclusion:

City Hotel Cancellation Rate: ~42%

Resort Hotel Cancellation Rate: ~28%

Insight: City hotels are more prone to cancellations, suggesting a need for better booking commitment strategies (like prepayment or incentives for confirmed stays).

Business Impact: Helps improve forecast accuracy, revenue management, and operational planning.
Conclusion:

City Hotel Cancellation Rate: 30.04%

Resort Hotel Cancellation Rate: 23.48%

Insight: City hotels experience ~6.5% higher cancellations than resort hotels.

Business Impact: Elite Hotels can enhance profitability by adjusting pricing strategies and cancellation policies based on these behavioral differences.

**Quetion 2 - What is the average ADR per market segment?**

In [ ]:
df.groupby('market_segment')['adr'].mean().sort_values(ascending=False)


Interpretation:

The highest ADRs are from Online TA (118.17) and Direct bookings (116.58) — showing that individual travelers tend to pay higher room rates.

Aviation bookings (crew or airline partnerships) have moderate ADR (100.17) — likely due to negotiated rates.

Groups (74.86) and Corporate (68.15) segments show discounted ADRs, reflecting bulk or contract pricing.

Complementary stays and Undefined segments contribute negligible revenue, as expected.
Conclusion:

Top Segments by ADR: Online TA (118.17) and Direct (116.58)

Lowest Segments by ADR: Complementary (3.05) and Undefined (15.00)

Insight: The hotel’s highest revenue potential lies in individual leisure travelers booking via online or direct channels.

Business Impact: Optimizing direct booking campaigns and reducing OTA dependency can significantly improve profit margins while maintaining occupancy.

**Question3: What is the relationship between lead time and cancellation rate?**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.scatterplot(x='lead_time', y='is_canceled', data=df, alpha=0.3)
plt.title('Relationship Between Lead Time and Cancellation Rate')
plt.xlabel('Lead Time (days)')
plt.ylabel('Cancellation (0 = Not Canceled, 1 = Canceled)')
plt.show()


In [ ]:
correlation = df['lead_time'].corr(df['is_canceled'])
print(correlation)


Interpretation

The positive correlation (0.18) means that as lead time increases, the likelihood of cancellation also increases — but the relationship is moderate, not very strong.

This suggests that bookings made far in advance are somewhat more prone to cancellation, though other factors (like market segment, deposit type, etc.) also play a role.

Summary Statement for Report

The correlation between lead time and cancellation rate is 0.18, indicating a moderate positive relationship. Guests who book further in advance tend to cancel more often. This insight can help management refine cancellation policies, adjust forecasts, and develop tailored pricing strategies for early bookings.

**Question4: Which distribution channel has the highest number of bookings?**

In [ ]:
channel_counts = df['distribution_channel'].value_counts()
channel_percent = df['distribution_channel'].value_counts(normalize=True) * 100

print(channel_counts)
print(channel_percent)


Interpretation

The TA/TO channel clearly dominates, accounting for ~79% of all bookings.

Direct bookings form a smaller but valuable segment (15%), indicating potential for growth through better marketing and loyalty initiatives.

Corporate and GDS channels are niche but stable, contributing a small share of high-value or bulk bookings.

Summary for Report

The analysis shows that the TA/TO (Travel Agent/Tour Operator) distribution channel contributes nearly 79% of total bookings, followed by Direct (15%) and Corporate (6%) channels. This indicates strong dependency on third-party platforms. To improve profitability and brand control, Elite Hotels should expand direct digital marketing and corporate engagement, reducing reliance on intermediaries over time.

**Question 5: What is the average number of previous cancellations by hotel type?**

In [ ]:
avg_prev_cancellations = df.groupby('hotel')['previous_cancellations'].mean()
print(avg_prev_cancellations)


Interpretation

Guests staying at City Hotels have a slightly higher average of previous cancellations compared to Resort Hotel guests.

Although the values are small (less than 0.04 on average), the trend is consistent — urban travelers (City Hotels) are more likely to have a cancellation history.

This pattern can indicate less stable or short-notice travel plans, often related to business or city travel.

**Question6: What is the trend of ADR (Average Daily Rate) over the years?**

In [ ]:
adr_trend = df.groupby('arrival_date_year')['adr'].mean().reset_index()
print(adr_trend)

# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(adr_trend['arrival_date_year'], adr_trend['adr'], marker='o', linestyle='-')
plt.title('Average Daily Rate (ADR) Trend Over Years')
plt.xlabel('Year of Arrival')
plt.ylabel('Average ADR')
plt.grid(True)
plt.show()


Interpretation:

There is a consistent year-on-year increase in ADR from 2015 → 2017.

The ADR grew by approximately:

+10.2% from 2015 to 2016, and

+16.9% from 2016 to 2017.

This reflects an improvement in revenue per room, suggesting either:

Increased demand and occupancy,

Better revenue optimization strategies, or

Shift toward higher-value customer segments.

Conclusion

The Average Daily Rate (ADR) increased from ₹92.16 in 2015 to ₹118.71 in 2017, reflecting a steady growth of ~29% over three years.
This positive trend demonstrates effective revenue management and increasing market value, positioning the hotels well for premium pricing strategies in future years.

**Question7: Which month has the highest revenue?**

In [ ]:
# Calculate total stay nights and total revenue
df['total_stay_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
df['revenue'] = df['adr'] * df['total_stay_nights']

# Group by month and sum revenue
monthly_revenue = df.groupby('arrival_date_month')['revenue'].sum().sort_values(ascending=False)
print(monthly_revenue)


In [ ]:
monthly_revenue.plot(kind='bar', figsize=(10,5), title='Total Monthly Revenue', ylabel='Revenue', xlabel='Month')
plt.show()


Interpretation:

August generates the highest total revenue (≈7.24M), confirming it as the peak month for both bookings and revenue.

The top three months — August, July, and June — together contribute over 45% of total annual revenue, indicating a strong summer seasonality pattern.

Lower revenue months (like January and November) align with off-season travel periods, suggesting potential for discounted promotions or group packages.

Summary

The month of August recorded the highest total revenue (7.24M), followed by July (5.86M) and June (3.49M).
This indicates that the summer period is the most profitable season, driven by increased bookings and higher ADRs.
Targeted marketing, dynamic pricing, and operational optimization during these months can maximize annual profitability.

**Question8: What is the impact of special requests on ADR?**

In [ ]:
special_req_impact = df.groupby('total_of_special_requests')['adr'].mean().reset_index()
print(special_req_impact)

# Optional visualization
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(special_req_impact['total_of_special_requests'], special_req_impact['adr'], marker='o')
plt.title('Impact of Special Requests on ADR')
plt.xlabel('Number of Special Requests')
plt.ylabel('Average Daily Rate (ADR)')
plt.grid(True)
plt.show()


In [ ]:
correlation = df['total_of_special_requests'].corr(df['adr'])
print(correlation)


**Question 9: What is the average stay duration for repeated guests versus new guests?**

In [ ]:
df['total_stay'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

stay_by_guest_type = df.groupby('is_repeated_guest')['total_stay'].mean()
stay_by_guest_type


Result is_repeated_guest
0    3.699932
1    1.927086
Interpretation

New guests stay significantly longer (≈3.7 nights) than repeated guests (≈1.9 nights).

This suggests that first-time visitors tend to book longer vacations or extended stays, possibly for exploration or leisure.

Repeated guests stay shorter, which is typical for:

Business travelers

Frequent short city visits

Returning guests who are familiar and require shorter stays

**Question 10: Which room type has the highest number of bookings?**

In [ ]:
room_counts = df['reserved_room_type'].value_counts()
print(room_counts)


Result reserved_room_type
A    56552
D    17398
E     6049
F     2823
G     2052
B      999
C      915
H      596
L        6
P        6
Room Type A with 56,552 bookings
Room Type A is overwhelmingly the most popular, with nearly 65% of all bookings.

Interpretation
Room Type A is the clear guest favorite, far ahead of other room categories.
Room Types D and E show moderate demand.
Room Types L and P have extremely low bookings, indicating rare or premium/special use rooms.

**Advanced Level Questions**

**Quetion 1 : What factors significantly impact the cancellation rate?**

In [ ]:
# -----------------------------
# 1. Import Libraries
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# -----------------------------
# 2. Load Dataset
# -----------------------------
df = pd.read_csv("/kaggle/input/hotel-booking-demand/hotel_bookings.csv")

# -----------------------------
# 3. Select Features for Model
# -----------------------------
features = [
    'lead_time',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'booking_changes',
    'adr',
    'total_of_special_requests',
    'deposit_type',
    'customer_type',
    'market_segment'
]

X = df[features]
y = df['is_canceled']

# -----------------------------
# 4. Separate Numerical & Categorical Columns
# -----------------------------
numeric_features = [
    'lead_time',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'booking_changes',
    'adr',
    'total_of_special_requests'
]

categorical_features = ['deposit_type', 'customer_type', 'market_segment']

# -----------------------------
# 5. Preprocessing Pipeline
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Transform X
X_processed = preprocessor.fit_transform(X)

# -----------------------------
# 6. Train/Test Split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

# -----------------------------
# 7. Train Logistic Regression Model
# -----------------------------
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# -----------------------------
# 8. Model Evaluation
# -----------------------------
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# -----------------------------
# 9. Show Coefficients (Feature Importance)
# -----------------------------
# Get feature names after OneHotEncoding
cat_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(cat_cols)

coef_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance (Sorted):")
print(coef_df)


Business Insight
OTA + long lead-time + non-refundable = high cancellation cluster
Past behavior is the strongest indicator of future cancellations.

**Quetion 2: How does the ADR vary with the number of adults, children, and babies?**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

# Select features
df_clean = df[['adults', 'children', 'babies', 'adr']].dropna()

X = df_clean[['adults', 'children', 'babies']]
y = df_clean['adr']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# R² score
print("R² Score:", r2_score(y_test, y_pred))

# Coefficients
coefficients = pd.DataFrame({
    'Feature': ['adults', 'children', 'babies'],
    'Coefficient': model.coef_
})
print("\nCoefficients:")
print(coefficients)

# Statsmodels summary
X_sm = sm.add_constant(X)
sm_model = sm.OLS(y, X_sm).fit()
print("\nStatsmodels Summary:")
print(sm_model.summary())


R² Score = 0.1635 (≈16%)
This means:
Only 16% of variation in ADR is explained by the number of adults, children, and babies.
Guest composition affects ADR slightly, but many other factors matter more (hotel type, seasonality, room type, deposit type, etc.).
This is expected — ADR is complex and depends on pricing strategy, not only number of people
What does this mean?

Children have the highest impact on ADR, almost double that of adults.
→ Likely because families book larger or premium rooms.

Adults still increase ADR, but less significantly.

Babies increase ADR minimally, as hotels often don’t charge much extra for infants.

**Quetion 3 : What is the impact of booking changes on guest satisfaction as indicated by special requests?**

In [ ]:
# Calculate correlation between booking changes and special requests
correlation = df['booking_changes'].corr(df['total_of_special_requests'])

print("Correlation between booking_changes and total_of_special_requests:", correlation)


Interpretation of the Result
A correlation of 0.0528 indicates a very weak positive relationship between:
booking_changes
total_of_special_requests
✔ What this means:
Guests who change their bookings only very slightly tend to make more special requests.
But the relationship is so small that it is practically insignificant.
In reality, booking changes and special requests are mostly independent behaviors.

Business Impact
1️⃣ Booking Flexibility Does Not Increase Special Request Workload

Since the correlation is extremely low, hotels can confidently allow free booking modifications without worrying about:

more operational burden

increase in request handling time

more pressure on housekeeping or front desk

2️⃣ Better Customer Satisfaction

Flexible booking policies improve guest satisfaction without increasing special-request demands.

3️⃣ Operational Efficiency

Special requests (like extra beds, room views, cots, late checkouts) are not driven by booking changes.

This allows separate planning for:

reservation changes team

front-desk and housekeeping teams

4️⃣ Policy Optimization

Hotels can:

provide self-service booking modification options

offer flexible change windows

avoid unnecessary fees for changes

Since changes have minimal impact on staff workload, this improves guest experience at low cost.

Quetion 4 : What is the seasonal impact on booking cancellations?


Cancellation Rate= Total Bookings/Number of Cancelled Bookings
	


In [ ]:
import pandas as pd

# Calculate monthly cancellation rate
monthly_cancel_rate = df.groupby('arrival_date_month')['is_canceled'].mean().sort_values()

print("Monthly Cancellation Rates:")
print(monthly_cancel_rate)

# Convert months to seasons
season_map = {
    'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
    'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
    'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
    'September': 'Fall', 'October': 'Fall', 'November': 'Fall'
}

df['season'] = df['arrival_date_month'].map(season_map)

# Calculate seasonal cancellation rate
season_cancel_rate = df.groupby('season')['is_canceled'].mean().sort_values()

print("\nSeasonal Cancellation Rates:")
print(season_cancel_rate)


Cancellations peak during Summer and are lowest during Winter. Spring and Fall lie in between, with Spring slightly higher due to flexible leisure travel patterns.
Seasonal trends show a clear pattern where summer months experience the highest cancellation rates (38–41%), likely due to leisure travel fluctuations and peak pricing.
Winter months have the lowest cancellation rates (33%), as travel during the holiday period tends to be more fixed.
Understanding these seasonal variations helps hotels optimize overbooking, pricing, staffing, and customer communication strategies, ultimately reducing revenue loss from cancellations.

Quetion 5 : How does the booking lead time distribution vary between different market segments?


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 6))

sns.boxplot(
    data=df,
    x='market_segment',
    y='lead_time'
)

plt.title("Lead Time Distribution by Market Segment")
plt.xlabel("Market Segment")
plt.ylabel("Lead Time (Days)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
